In [125]:
from tensorflow import keras
# import keras
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import load_model
from sklearn.utils import class_weight

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
from collections import Counter
import numpy as np
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.utils.multiclass import unique_labels

In [126]:
import cv2
import tensorflow as tf
import os

In [127]:
# Load the cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+"haarcascade_frontalface_default.xml")


In [128]:
path=r"C:\Users\ayon5\OneDrive\Desktop\emotion recognition\archive\train"



In [129]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import utils
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from IPython.display import SVG, Image
from livelossplot.inputs.tf_keras import PlotLossesCallback
import tensorflow as tf
print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.11.0


In [130]:
train_path =r"C:\Users\ayon5\OneDrive\Desktop\emotion recognition\archive\train"
test_path = r"C:\Users\ayon5\OneDrive\Desktop\emotion recognition\archive\test"

In [137]:
batch_size= 32
scale = 1/255
target_size = (224,224)

train_gen = ImageDataGenerator(rescale=scale)
test_gen = ImageDataGenerator(rescale=scale)

train_generator= train_gen.flow_from_directory(train_path, batch_size=batch_size,target_size=target_size)
test_generator = test_gen.flow_from_directory(test_path,target_size=target_size)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [132]:
model = Sequential()
num_classes=7

# First CONV-ReLU Layer
model.add(Conv2D(32, (7, 7), padding = 'same', input_shape = (target_size[0], target_size[1],3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second CONV-ReLU Layer
model.add(Conv2D(64, (5, 5), padding = "same"))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.4))

model.add(Conv2D(96, (3, 3), padding="same"))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, (3, 3), padding="same",kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.4))

model.add(Conv2D(256, (3, 3), padding="same",kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding="same",kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())

model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.4))


# Final Dense Layer
model.add(Dense(num_classes,activation='softmax',name='hafizzo'))

print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 224, 224, 32)      4736      
                                                                 
 batch_normalization_15 (Bat  (None, 224, 224, 32)     128       
 chNormalization)                                                
                                                                 
 activation_17 (Activation)  (None, 224, 224, 32)      0         
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 112, 112, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 112, 112, 64)      51264     
                                                                 
 batch_normalization_16 (Bat  (None, 112, 112, 64)    

In [140]:
             
checkpoint = ModelCheckpoint("cnn_emotiondetectio.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)


# we put our call backs into a callback list
callbacks = [ checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(learning_rate=0.0001),
              metrics = ['accuracy'])


epochs = 10

history= model.fit(   
    train_generator,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = test_generator)

Epoch 1/10
121/898 [===>..........................] - ETA: 29:25 - loss: 7.3803 - accuracy: 0.2417

KeyboardInterrupt: 

In [142]:
print(history.history.keys())
# Plotting our loss charts
import matplotlib.pyplot as plt

history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)

line1 = plt.plot(epochs, val_loss_values, label='Validation/Test Loss')
line2 = plt.plot(epochs, loss_values, label='Training Loss')
plt.setp(line1, linewidth=2.0, marker = '+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker = '4', markersize=10.0)
plt.xlabel('Epochs') 
plt.ylabel('Loss')
plt.grid(True)
plt.legend()
plt.savefig('Loss_stack.png', dpi = 1200)
plt.show()

False


In [ ]:
# Plotting our accuracy charts
import matplotlib.pyplot as plt

history_dict = history.history
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(loss_values) + 1)
line1 = plt.plot(epochs, val_acc_values, label='Validation/Test Accuracy')
line2 = plt.plot(epochs, acc_values, label='Training Accuracy')
plt.setp(line1, linewidth=2.0, marker = '+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker = '4', markersize=10.0)
plt.xlabel('Epochs') 
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()
plt.savefig('Accuracy_stack.png', dpi = 1200)
plt.show()

In [ ]:
import seaborn as sr
validation_path=validation_data_dir ="/home/kashraf/felix_hd/data_gen_may_2021/Audio_topomaps_June21/stack/validation/"
validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())
nb_train_samples = 29328
nb_validation_samples = 6192


#Confution Matrix and Classification Report
Y_pred = model.predict(validation_generator, nb_validation_samples // batch_size)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
import pandas as pd
conf_mat=confusion_matrix(validation_generator.classes,y_pred,normalize='true')

conf_df=pd.DataFrame(conf_mat, index=["CL-1","CL-2","CL-3","CL-4"], columns=["CL-1","CL-2","CL-3","CL-4"])
# print("\nFace  accuracy =",accuracy)
# print("\n Face recognition report: \n",report)
fig=plt.figure(figsize=(8,6))
sr.heatmap(conf_df,annot=True,cmap="Blues")
# plt.title("Confusion matrix")
plt.xlabel("PREDICTED LABEL")
plt.ylabel("TRUE LABEL")
plt.savefig("Conf_mat_stack_valid")